# Simple q-learning agent with experience replay

We re-write q-learning algorithm using _agentnet_ - a helper for lasagne that implements some RL techniques.

In [1]:
! pip install --upgrade https://github.com/yandexdataschool/AgentNet/archive/master.zip

     - 11.7MB 73.4MB/s
Requirement already up-to-date: six in ./anaconda3/envs/py34/lib/python3.4/site-packages (from agentnet==0.10.6)
Requirement already up-to-date: lasagne in ./AgentNet/src/lasagne (from agentnet==0.10.6)
Requirement already up-to-date: theano>=0.8.2 in ./anaconda3/envs/py34/lib/python3.4/site-packages (from agentnet==0.10.6)
Requirement already up-to-date: numpy>=1.9 in ./anaconda3/envs/py34/lib/python3.4/site-packages (from agentnet==0.10.6)
Requirement already up-to-date: scipy>=0.14 in ./anaconda3/envs/py34/lib/python3.4/site-packages (from theano>=0.8.2->agentnet==0.10.6)
  Found existing installation: agentnet 0.10.6
    Uninstalling agentnet-0.10.6:
      Successfully uninstalled agentnet-0.10.6
  Running setup.py install for agentnet ... - \ | done


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
%env THEANO_FLAGS='floatX=float32'

#XVFB will be launched if you run on a server
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

env: THEANO_FLAGS='floatX=float32'
bash: ../xvfb: No such file or directory
env: DISPLAY=:1


### Experiment setup
* Here we simply load the game and check that it works

In [3]:
import gym
make_env = lambda: gym.make("LunarLander-v2")

env = gym.make("LunarLander-v2")
env.reset()

state_shape = env.observation_space.shape
n_actions = env.action_space.n

In [4]:
print(state_shape)

(8,)


In [5]:
# plt.imshow(env.render("rgb_array"))
# del env

In [ ]:
net = lasagne.layers.InputLayer(shape=(None, 3, 32, 32), input_var=input_X)

# Neural Network body

In [6]:
import lasagne
from lasagne.layers import *
from lasagne.nonlinearities import *
# from lasagne.nonlinearities import relu


#image observation at current tick goes here, shape = (sample_i,x,y,color)
# inp_layer = InputLayer((None, 3, 8, 8))
observation_layer = InputLayer((None, ) + state_shape)

net = DenseLayer(observation_layer, 200, nonlinearity=elu)
net = DenseLayer(net, 200, nonlinearity=elu)
# nn = DenseLayer(nn, 200)

#a layer that predicts Qvalues
qvalues_layer = DenseLayer(net, num_units=n_actions,
                           nonlinearity=None,name="q-values")

Picking actions is done by yet another layer, that implements $ \epsilon$ -greedy policy

In [7]:
from agentnet.resolver import EpsilonGreedyResolver

In [8]:
action_layer = EpsilonGreedyResolver(qvalues_layer)

#set starting epsilon
action_layer.epsilon.set_value(np.float32(0.05))

### Agent

We define an agent entirely composed of a lasagne network:
* Observations as InputLayer(s)
* Actions as intermediate Layer(s)
* `policy_estimators` is "whatever else you want to keep track of"

Each parameter can be either one layer or a list of layers

In [9]:
from agentnet.agent import Agent
agent = Agent(observation_layers=observation_layer,
              action_layers=action_layer,
              policy_estimators=qvalues_layer,)

In [10]:
#Since it's a single lasagne network, one can get it's weights, output, etc
weights = lasagne.layers.get_all_params(action_layer,trainable=True)
weights

[W, b, W, b, q-values.W, q-values.b]

# Create and manage a pool of atari sessions to play with

* To make training more stable, we shall have an entire batch of game sessions each happening independent of others
* Why several parallel agents help training: http://arxiv.org/pdf/1602.01783v1.pdf
* Alternative approach: store more sessions: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf

In [11]:
from agentnet.experiments.openai_gym.pool import EnvPool
pool = EnvPool(agent,make_env,n_games=1,max_size=10000)

In [12]:
%%time
#interact for 7 ticks
obs_log,action_log,reward_log,_,_,_  = pool.interact(5)


print('actions:',action_log)
print('rewards:',reward_log)

actions: [[1 1 3 1 0]]
rewards: [[-1.81280494 -2.10888041 -1.92483021 -2.06786121  0.        ]]
CPU times: user 28 ms, sys: 4 ms, total: 32 ms
Wall time: 23.7 ms


In [13]:
#we'll train on rollouts of 10 steps (required by n-step algorithms and rnns later)
SEQ_LENGTH=10

#load first sessions (this function calls interact and stores sessions in the pool)

for _ in range(100):
    pool.update(SEQ_LENGTH)

# q-learning

We shall now define a function that replays recent game sessions and updates network weights

In [14]:
#get agent's Qvalues obtained via experience replay
replay = pool.experience_replay.sample_session_batch(100)
qvalues_seq = agent.get_sessions(
    replay,
    session_length=SEQ_LENGTH,
    experience_replay=True,
)[-1]



In [15]:
#loss for Qlearning = (Q(s,a) - (r+gamma*Q(s',a_max)))^2, like you implemented before in lasagne.

from agentnet.learning import qlearning
elwise_mse_loss = qlearning.get_elementwise_objective(qvalues_seq,
                                                      replay.actions[0],
                                                      replay.rewards,
                                                      replay.is_alive,
                                                      gamma_or_gammas=0.99,
                                                      n_steps=1,)

#compute mean loss over "alive" fragments
loss = elwise_mse_loss.sum() / replay.is_alive.sum()

In [16]:
#get weight updates
updates = lasagne.updates.adam(loss,weights,learning_rate=1e-4)

#compile train function
import theano
train_step = theano.function([],loss,updates=updates)

# Training loop

In [17]:
epoch_counter = 1 #starting epoch
rewards = {} #full game rewards
target_score = 0

In [18]:
from tqdm import trange

for i in trange(10500):    
    
    #play
    for _ in range(5):
        pool.update(SEQ_LENGTH,append=True)
    
    #train
    train_step()
    
    #update epsilon
    epsilon = 0.05 + 0.95*np.exp(-epoch_counter/1000.)
    action_layer.epsilon.set_value(np.float32(epsilon))
    
    #play a few games for evaluation
    if epoch_counter%50==0:
        rewards[epoch_counter] = np.mean(pool.evaluate(n_games=3,record_video=False))
        print("iter=%i\tepsilon=%.3f"%(epoch_counter,action_layer.epsilon.get_value(),))
        print("Current score(mean over %i) = %.3f"%(3,np.mean(rewards[epoch_counter])))
    
        if rewards[epoch_counter] >= target_score:
            print("You win!")
            break

    
    epoch_counter  +=1

    
# Time to drink some coffee!

  0%|          | 51/10500 [00:03<11:20, 15.34it/s]

Episode finished after 102 timesteps with reward=-130.88768128867397
Episode finished after 67 timesteps with reward=-116.56477484809187
Episode finished after 71 timesteps with reward=-213.61946112192328
iter=50	epsilon=0.954
Current score(mean over 3) = -153.691


  1%|          | 99/10500 [00:06<11:32, 15.02it/s]

Episode finished after 104 timesteps with reward=-137.68896482463805
Episode finished after 90 timesteps with reward=-183.00277077741316
Episode finished after 95 timesteps with reward=-153.6674467105105
iter=100	epsilon=0.910
Current score(mean over 3) = -158.120


  1%|▏         | 149/10500 [00:10<11:53, 14.50it/s]

...

Episode finished after 976 timesteps with reward=-400.878857921913
iter=10450	epsilon=0.050
Current score(mean over 3) = -279.669


100%|█████████▉| 10499/10500 [20:33<00:00,  8.51it/s]

Episode finished after 312 timesteps with reward=-292.5437785368547
Episode finished after 447 timesteps with reward=-190.57506065380454


100%|██████████| 10500/10500 [20:34<00:00,  8.50it/s]

Episode finished after 392 timesteps with reward=-222.733239235747
iter=10500	epsilon=0.050
Current score(mean over 3) = -235.284


In [ ]:
from pandas import ewma
iters,session_rewards=zip(*sorted(rewards.items(),key=lambda (k,v):k))
plt.plot(iters,ewma(np.array(session_rewards),span=10))

In [ ]:
final_reward = pool.evaluate(n_games=10,save_path="./records",record_video=True)

print("average reward:",final_reward)

video_names = list(filter(lambda s:s.endswith(".mp4"),os.listdir("./records/")))

for video_name in video_names:
    HTML("""
    <video width="640" height="480" controls>
      <source src="{}" type="video/mp4">
    </video>
    """.format("./records/"+video_name)) #this may or may not be _last_ video. Try other indices